## Итоговый датафрейм

Собираем воедино, все, что раньше было разбросано по частям

### Собираем большой датафрейм с парами сообщение - компания

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = '../data/messages/messages_info.csv'
data_messages = pd.read_csv(path, index_col = 0)
data_messages['date'] = pd.to_datetime(data_messages['date'])
data_messages.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,banana_react_cnt,poop_react_cnt,rofl_smile_react_cnt,clown_smile_react_cnt,devil_smile_react_cnt,hundred_react_cnt,head_arm_smile_react_cnt,eyes_react_cnt,company,ticker
0,1675,3,22642,2023-10-06 16:37:06,все больше компаний рф заявляют о планах по ip...,65009,84,301,14,0,...,0,0,0,0,0,0,0,0,crystal,KLVZ
1,1474,3,22861,2023-10-31 11:41:23,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0,0,0,0,0,0,0,0,crystal,KLVZ
2,758,3,23707,2024-02-20 18:20:41,📺 алкогольная группа «кристалл» уже в этот чет...,69592,119,146,86,0,...,0,0,0,0,0,0,0,0,crystal,KLVZ


In [3]:
path = '../data/quotes/quotes_prep.csv'
ipo_data = pd.read_csv(path, index_col = 0)
ipo_data['ipo_dt'] = pd.to_datetime(ipo_data['ipo_dt']).apply(lambda x: x.date())
ipo_data.head(3)

,ticker,name_ru,ipo_dt,ipo_price,1d_price,1m_price,3m_price,6m_price
0,KLVZ,Кристалл,2024-02-22,9.5,-0.147,NaN,NaN,NaN
1,DIAS,Диасофт,2024-02-13,4500.0,0.400,NaN,NaN,NaN
2,DELI,Делимобиль,2024-06-02,265.0,0.068,0.08,NaN,NaN


In [4]:
path = '../data/messages/tonality.csv'
tonality_data = pd.read_csv(path, index_col = 0)
tonality_data.head(3)

,message_id,text,batch_num,neutral_score,positive_score,negative_score,tonality_class
0,1675,все больше компаний рф заявляют о планах по ip...,0,1.812892,0.167650,-1.352255,NEUTRAL
1,1474,🇷🇺 тема ipo на российском фондовом рынке сегод...,1,-0.059815,-1.039290,1.365424,NEGATIVE
2,758,📺 алкогольная группа «кристалл» уже в этот чет...,2,1.851411,0.183083,-1.381232,NEUTRAL


In [5]:
path = '../data/quotes/rvi_prep.csv'
rvi_data = pd.read_csv(path, index_col = 0)
rvi_data['date'] = pd.to_datetime(rvi_data['date']).apply(lambda x: x.date())
rvi_data.head(3)

,date,price,open,max,min,delta_perc,delta_open,delta_price
0,2024-04-30,20.73,20.04,21.32,19.88,0.0313,-0.15,0.63
1,2024-04-29,20.10,20.19,21.56,19.83,0.0912,-0.70,1.68
2,2024-04-26,18.42,20.89,20.90,18.28,-0.1006,-0.47,-2.06


In [6]:
data = pd.merge(data_messages, ipo_data, how='inner', left_on='ticker', right_on='ticker')
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,eyes_react_cnt,company,ticker,name_ru,ipo_dt,ipo_price,1d_price,1m_price,3m_price,6m_price
0,1675,3,22642,2023-10-06 16:37:06,все больше компаний рф заявляют о планах по ip...,65009,84,301,14,0,...,0,crystal,KLVZ,Кристалл,2024-02-22,9.5,-0.147,NaN,NaN,NaN
1,1474,3,22861,2023-10-31 11:41:23,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0,crystal,KLVZ,Кристалл,2024-02-22,9.5,-0.147,NaN,NaN,NaN
2,758,3,23707,2024-02-20 18:20:41,📺 алкогольная группа «кристалл» уже в этот чет...,69592,119,146,86,0,...,0,crystal,KLVZ,Кристалл,2024-02-22,9.5,-0.147,NaN,NaN,NaN


In [7]:
data['date'] = data['date'].apply(lambda x: x.date())
data['days_before_ipo'] = data.apply(lambda x: (x['date'] - x['ipo_dt']).days, axis = 1)
data['text_len'] = data['text'].apply(lambda x: len(x))

In [8]:
data = pd.merge(data, rvi_data[['date', 'price', 'delta_price', 'delta_perc']], how = 'inner', left_on = 'date', right_on = 'date', suffixes=('', '_on_message_date'))
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,ipo_price,1d_price,1m_price,3m_price,6m_price,days_before_ipo,text_len,price,delta_price,delta_perc
0,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,9.5,-0.147,NaN,NaN,NaN,-114,1938,27.39,2.28,0.0908
1,9355,10,6521,2023-10-31,🚀 участвуем в ipo без участия в ipo\n\nделовые...,18533,15,76,6,0,...,9.5,-0.147,NaN,NaN,NaN,-114,1228,27.39,2.28,0.0908
2,9355,10,6521,2023-10-31,🚀 участвуем в ipo без участия в ipo\n\nделовые...,18533,15,76,6,0,...,4500.0,0.400,NaN,NaN,NaN,-105,1228,27.39,2.28,0.0908


In [9]:
data = pd.merge(data, rvi_data[['date', 'open', 'delta_open', 'delta_perc']], how = 'inner', left_on = 'date', right_on = 'date', suffixes=('', '_on_ipo_date'))
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,3m_price,6m_price,days_before_ipo,text_len,price,delta_price,delta_perc,open,delta_open,delta_perc_on_ipo_date
0,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,NaN,NaN,-114,1938,27.39,2.28,0.0908,25.57,-0.39,0.0908
1,9355,10,6521,2023-10-31,🚀 участвуем в ipo без участия в ipo\n\nделовые...,18533,15,76,6,0,...,NaN,NaN,-114,1228,27.39,2.28,0.0908,25.57,-0.39,0.0908
2,9355,10,6521,2023-10-31,🚀 участвуем в ipo без участия в ipo\n\nделовые...,18533,15,76,6,0,...,NaN,NaN,-105,1228,27.39,2.28,0.0908,25.57,-0.39,0.0908


In [10]:
data = pd.merge(data, tonality_data[['message_id','neutral_score', 'positive_score', 'negative_score']], how = 'inner', on = 'message_id')
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,text_len,price,delta_price,delta_perc,open,delta_open,delta_perc_on_ipo_date,neutral_score,positive_score,negative_score
0,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,1938,27.39,2.28,0.0908,25.57,-0.39,0.0908,-0.059815,-1.03929,1.365424
1,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,1938,27.39,2.28,0.0908,25.57,-0.39,0.0908,-0.059815,-1.03929,1.365424
2,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,1938,27.39,2.28,0.0908,25.57,-0.39,0.0908,-0.059815,-1.03929,1.365424


In [11]:
with pd.option_context("display.max_rows", 1000):
    display(list(data.columns))

['message_id',
 'channel_id',
 'tg_message_id',
 'date',
 'text',
 'views',
 'forwards',
 'thumb_up_react_cnt',
 'thumb_down_react_cnt',
 'mindblowing_react_cnt',
 'heart_react_cnt',
 'thinking_react_cnt',
 'crying_react_cnt',
 'fire_react_cnt',
 'clap_react_cnt',
 'whale_react_cnt',
 'champaign_react_cnt',
 'heart_fire_react_cnt',
 'okay_react_cnt',
 'smile_stars_react_cnt',
 'smiling_react_cnt',
 'cheers_react_cnt',
 'lovely_smile_react_cnt',
 'drunk_smile_react_cnt',
 'angry_smile_react_cnt',
 'shicked_smile_react_cnt',
 'moos_smile_react_cnt',
 'hotdog_react_cnt',
 'free_bird_react_cnt',
 'snowman_react_cnt',
 'banana_react_cnt',
 'poop_react_cnt',
 'rofl_smile_react_cnt',
 'clown_smile_react_cnt',
 'devil_smile_react_cnt',
 'hundred_react_cnt',
 'head_arm_smile_react_cnt',
 'eyes_react_cnt',
 'company',
 'ticker',
 'name_ru',
 'ipo_dt',
 'ipo_price',
 '1d_price',
 '1m_price',
 '3m_price',
 '6m_price',
 'days_before_ipo',
 'text_len',
 'price',
 'delta_price',
 'delta_perc',
 'open

In [12]:
sentiment_cols = ['views',
       'forwards', 'thumb_up_react_cnt', 'thumb_down_react_cnt',
       'mindblowing_react_cnt', 'heart_react_cnt', 'thinking_react_cnt',
       'crying_react_cnt', 'fire_react_cnt', 'clap_react_cnt',
       'whale_react_cnt', 'champaign_react_cnt', 'heart_fire_react_cnt',
       'okay_react_cnt', 'smile_stars_react_cnt', 'smiling_react_cnt',
       'cheers_react_cnt', 'lovely_smile_react_cnt', 'drunk_smile_react_cnt',
       'angry_smile_react_cnt', 'shicked_smile_react_cnt',
       'moos_smile_react_cnt', 'hotdog_react_cnt', 'free_bird_react_cnt',
       'snowman_react_cnt', 'banana_react_cnt', 'poop_react_cnt',
       'rofl_smile_react_cnt', 'clown_smile_react_cnt',
       'devil_smile_react_cnt', 'hundred_react_cnt',
       'head_arm_smile_react_cnt', 'eyes_react_cnt']

sentiment_cols_new = [item + '_x_positive' for item in sentiment_cols]

data[sentiment_cols_new] = data[sentiment_cols] * np.sign(data[sentiment_cols])
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,free_bird_react_cnt_x_positive,snowman_react_cnt_x_positive,banana_react_cnt_x_positive,poop_react_cnt_x_positive,rofl_smile_react_cnt_x_positive,clown_smile_react_cnt_x_positive,devil_smile_react_cnt_x_positive,hundred_react_cnt_x_positive,head_arm_smile_react_cnt_x_positive,eyes_react_cnt_x_positive
0,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0,0,0,0,0,0,0,0,0,0
1,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0,0,0,0,0,0,0,0,0,0
2,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
numeric_cols =[
 'views',
 'forwards',
 'thumb_up_react_cnt',
 'thumb_down_react_cnt',
 'mindblowing_react_cnt',
 'heart_react_cnt',
 'thinking_react_cnt',
 'crying_react_cnt',
 'fire_react_cnt',
 'clap_react_cnt',
 'whale_react_cnt',
 'champaign_react_cnt',
 'heart_fire_react_cnt',
 'okay_react_cnt',
 'smile_stars_react_cnt',
 'smiling_react_cnt',
 'cheers_react_cnt',
 'lovely_smile_react_cnt',
 'drunk_smile_react_cnt',
 'angry_smile_react_cnt',
 'shicked_smile_react_cnt',
 'moos_smile_react_cnt',
 'hotdog_react_cnt',
 'free_bird_react_cnt',
 'snowman_react_cnt',
 'banana_react_cnt',
 'poop_react_cnt',
 'rofl_smile_react_cnt',
 'clown_smile_react_cnt',
 'devil_smile_react_cnt',
 'hundred_react_cnt',
 'head_arm_smile_react_cnt',
 'eyes_react_cnt',
 'text_len',
 'positive_score',
 'views_x_positive',
 'forwards_x_positive',
 'thumb_up_react_cnt_x_positive',
 'thumb_down_react_cnt_x_positive',
 'mindblowing_react_cnt_x_positive',
 'heart_react_cnt_x_positive',
 'thinking_react_cnt_x_positive',
 'crying_react_cnt_x_positive',
 'fire_react_cnt_x_positive',
 'clap_react_cnt_x_positive',
 'whale_react_cnt_x_positive',
 'champaign_react_cnt_x_positive',
 'heart_fire_react_cnt_x_positive',
 'okay_react_cnt_x_positive',
 'smile_stars_react_cnt_x_positive',
 'smiling_react_cnt_x_positive',
 'cheers_react_cnt_x_positive',
 'lovely_smile_react_cnt_x_positive',
 'drunk_smile_react_cnt_x_positive',
 'angry_smile_react_cnt_x_positive',
 'shicked_smile_react_cnt_x_positive',
 'moos_smile_react_cnt_x_positive',
 'hotdog_react_cnt_x_positive',
 'free_bird_react_cnt_x_positive',
 'snowman_react_cnt_x_positive',
 'banana_react_cnt_x_positive',
 'poop_react_cnt_x_positive',
 'rofl_smile_react_cnt_x_positive',
 'clown_smile_react_cnt_x_positive',
 'devil_smile_react_cnt_x_positive',
 'hundred_react_cnt_x_positive',
 'head_arm_smile_react_cnt_x_positive',
 'eyes_react_cnt_x_positive']

for col in numeric_cols:
    data[col+'_div_by_days'] = data[col]/data['days_before_ipo']
    
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,free_bird_react_cnt_x_positive_div_by_days,snowman_react_cnt_x_positive_div_by_days,banana_react_cnt_x_positive_div_by_days,poop_react_cnt_x_positive_div_by_days,rofl_smile_react_cnt_x_positive_div_by_days,clown_smile_react_cnt_x_positive_div_by_days,devil_smile_react_cnt_x_positive_div_by_days,hundred_react_cnt_x_positive_div_by_days,head_arm_smile_react_cnt_x_positive_div_by_days,eyes_react_cnt_x_positive_div_by_days
0,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
1,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
2,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
with pd.option_context("display.max_rows", 1000):
    display(list(data.columns))

['message_id',
 'channel_id',
 'tg_message_id',
 'date',
 'text',
 'views',
 'forwards',
 'thumb_up_react_cnt',
 'thumb_down_react_cnt',
 'mindblowing_react_cnt',
 'heart_react_cnt',
 'thinking_react_cnt',
 'crying_react_cnt',
 'fire_react_cnt',
 'clap_react_cnt',
 'whale_react_cnt',
 'champaign_react_cnt',
 'heart_fire_react_cnt',
 'okay_react_cnt',
 'smile_stars_react_cnt',
 'smiling_react_cnt',
 'cheers_react_cnt',
 'lovely_smile_react_cnt',
 'drunk_smile_react_cnt',
 'angry_smile_react_cnt',
 'shicked_smile_react_cnt',
 'moos_smile_react_cnt',
 'hotdog_react_cnt',
 'free_bird_react_cnt',
 'snowman_react_cnt',
 'banana_react_cnt',
 'poop_react_cnt',
 'rofl_smile_react_cnt',
 'clown_smile_react_cnt',
 'devil_smile_react_cnt',
 'hundred_react_cnt',
 'head_arm_smile_react_cnt',
 'eyes_react_cnt',
 'company',
 'ticker',
 'name_ru',
 'ipo_dt',
 'ipo_price',
 '1d_price',
 '1m_price',
 '3m_price',
 '6m_price',
 'days_before_ipo',
 'text_len',
 'price',
 'delta_price',
 'delta_perc',
 'open

In [15]:
data = data.query('days_before_ipo > 0').reset_index(drop=True)
data.head(3)

,message_id,channel_id,tg_message_id,date,text,views,forwards,thumb_up_react_cnt,thumb_down_react_cnt,mindblowing_react_cnt,...,free_bird_react_cnt_x_positive_div_by_days,snowman_react_cnt_x_positive_div_by_days,banana_react_cnt_x_positive_div_by_days,poop_react_cnt_x_positive_div_by_days,rofl_smile_react_cnt_x_positive_div_by_days,clown_smile_react_cnt_x_positive_div_by_days,devil_smile_react_cnt_x_positive_div_by_days,hundred_react_cnt_x_positive_div_by_days,head_arm_smile_react_cnt_x_positive_div_by_days,eyes_react_cnt_x_positive_div_by_days
0,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1474,3,22861,2023-10-31,🇷🇺 тема ipo на российском фондовом рынке сегод...,63078,63,251,19,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
to_drop = ['message_id',
 'channel_id',
 'tg_message_id',
 'date',
 'text',
 'company',
 'ticker',
 'name_ru',
 'ipo_dt',
 'ipo_price',
 'neutral_score',
 'negative_score']

In [17]:
path = '../data/final/pairs.csv'
data.drop(to_drop, axis = 1).to_csv(path)

### Аггрегированные данные

In [18]:
import sys
sys.path.insert(1, '../analytics/pipeline')

import pipeline as pl

In [19]:
cols = [
 'views',
 'forwards',
 'thumb_up_react_cnt',
 'thumb_down_react_cnt',
 'mindblowing_react_cnt',
 'heart_react_cnt',
 'thinking_react_cnt',
 'crying_react_cnt',
 'fire_react_cnt',
 'clap_react_cnt',
 'whale_react_cnt',
 'champaign_react_cnt',
 'heart_fire_react_cnt',
 'okay_react_cnt',
 'smile_stars_react_cnt',
 'smiling_react_cnt',
 'cheers_react_cnt',
 'lovely_smile_react_cnt',
 'drunk_smile_react_cnt',
 'angry_smile_react_cnt',
 'shicked_smile_react_cnt',
 'moos_smile_react_cnt',
 'hotdog_react_cnt',
 'free_bird_react_cnt',
 'snowman_react_cnt',
 'banana_react_cnt',
 'poop_react_cnt',
 'rofl_smile_react_cnt',
 'clown_smile_react_cnt',
 'devil_smile_react_cnt',
 'hundred_react_cnt',
 'head_arm_smile_react_cnt',
 'eyes_react_cnt',
 'ticker',
 'days_before_ipo',
 'text_len',
 'price',
 'delta_price',
 'delta_perc',
 'open',
 'delta_open',
 'delta_perc_on_ipo_date',
 'positive_score',
 'neutral_score',
 'negative_score',
 'views_x_positive',
 'forwards_x_positive',
 'thumb_up_react_cnt_x_positive',
 'thumb_down_react_cnt_x_positive',
 'mindblowing_react_cnt_x_positive',
 'heart_react_cnt_x_positive',
 'thinking_react_cnt_x_positive',
 'crying_react_cnt_x_positive',
 'fire_react_cnt_x_positive',
 'clap_react_cnt_x_positive',
 'whale_react_cnt_x_positive',
 'champaign_react_cnt_x_positive',
 'heart_fire_react_cnt_x_positive',
 'okay_react_cnt_x_positive',
 'smile_stars_react_cnt_x_positive',
 'smiling_react_cnt_x_positive',
 'cheers_react_cnt_x_positive',
 'lovely_smile_react_cnt_x_positive',
 'drunk_smile_react_cnt_x_positive',
 'angry_smile_react_cnt_x_positive',
 'shicked_smile_react_cnt_x_positive',
 'moos_smile_react_cnt_x_positive',
 'hotdog_react_cnt_x_positive',
 'free_bird_react_cnt_x_positive',
 'snowman_react_cnt_x_positive',
 'banana_react_cnt_x_positive',
 'poop_react_cnt_x_positive',
 'rofl_smile_react_cnt_x_positive',
 'clown_smile_react_cnt_x_positive',
 'devil_smile_react_cnt_x_positive',
 'hundred_react_cnt_x_positive',
 'head_arm_smile_react_cnt_x_positive',
 'eyes_react_cnt_x_positive',
 'views_div_by_days',
 'forwards_div_by_days',
 'thumb_up_react_cnt_div_by_days',
 'thumb_down_react_cnt_div_by_days',
 'mindblowing_react_cnt_div_by_days',
 'heart_react_cnt_div_by_days',
 'thinking_react_cnt_div_by_days',
 'crying_react_cnt_div_by_days',
 'fire_react_cnt_div_by_days',
 'clap_react_cnt_div_by_days',
 'whale_react_cnt_div_by_days',
 'champaign_react_cnt_div_by_days',
 'heart_fire_react_cnt_div_by_days',
 'okay_react_cnt_div_by_days',
 'smile_stars_react_cnt_div_by_days',
 'smiling_react_cnt_div_by_days',
 'cheers_react_cnt_div_by_days',
 'lovely_smile_react_cnt_div_by_days',
 'drunk_smile_react_cnt_div_by_days',
 'angry_smile_react_cnt_div_by_days',
 'shicked_smile_react_cnt_div_by_days',
 'moos_smile_react_cnt_div_by_days',
 'hotdog_react_cnt_div_by_days',
 'free_bird_react_cnt_div_by_days',
 'snowman_react_cnt_div_by_days',
 'banana_react_cnt_div_by_days',
 'poop_react_cnt_div_by_days',
 'rofl_smile_react_cnt_div_by_days',
 'clown_smile_react_cnt_div_by_days',
 'devil_smile_react_cnt_div_by_days',
 'hundred_react_cnt_div_by_days',
 'head_arm_smile_react_cnt_div_by_days',
 'eyes_react_cnt_div_by_days',
 'text_len_div_by_days',
 'positive_score_div_by_days',
 'views_x_positive_div_by_days',
 'forwards_x_positive_div_by_days',
 'thumb_up_react_cnt_x_positive_div_by_days',
 'thumb_down_react_cnt_x_positive_div_by_days',
 'mindblowing_react_cnt_x_positive_div_by_days',
 'heart_react_cnt_x_positive_div_by_days',
 'thinking_react_cnt_x_positive_div_by_days',
 'crying_react_cnt_x_positive_div_by_days',
 'fire_react_cnt_x_positive_div_by_days',
 'clap_react_cnt_x_positive_div_by_days',
 'whale_react_cnt_x_positive_div_by_days',
 'champaign_react_cnt_x_positive_div_by_days',
 'heart_fire_react_cnt_x_positive_div_by_days',
 'okay_react_cnt_x_positive_div_by_days',
 'smile_stars_react_cnt_x_positive_div_by_days',
 'smiling_react_cnt_x_positive_div_by_days',
 'cheers_react_cnt_x_positive_div_by_days',
 'lovely_smile_react_cnt_x_positive_div_by_days',
 'drunk_smile_react_cnt_x_positive_div_by_days',
 'angry_smile_react_cnt_x_positive_div_by_days',
 'shicked_smile_react_cnt_x_positive_div_by_days',
 'moos_smile_react_cnt_x_positive_div_by_days',
 'hotdog_react_cnt_x_positive_div_by_days',
 'free_bird_react_cnt_x_positive_div_by_days',
 'snowman_react_cnt_x_positive_div_by_days',
 'banana_react_cnt_x_positive_div_by_days',
 'poop_react_cnt_x_positive_div_by_days',
 'rofl_smile_react_cnt_x_positive_div_by_days',
 'clown_smile_react_cnt_x_positive_div_by_days',
 'devil_smile_react_cnt_x_positive_div_by_days',
 'hundred_react_cnt_x_positive_div_by_days',
 'head_arm_smile_react_cnt_x_positive_div_by_days',
 'eyes_react_cnt_x_positive_div_by_days']

In [20]:
cols = pl.nonsense_drop(data[cols], cutoff = 0.1)

In [21]:
cols_ = cols + ['ticker']

In [22]:
agg_dict = {}
for col in cols:
    agg_dict[col] = 'mean'

In [23]:
agg_dict = {}
for col in cols:
    agg_dict[col] = 'mean'
    
one_day_mean_data = data.query('days_before_ipo < 2')[cols_].groupby('ticker').agg(agg_dict).reset_index()
one_week_mean_data = data.query('days_before_ipo >= 2 and days_before_ipo < 8')[cols_].groupby('ticker').agg(agg_dict).reset_index()
one_month_mean_data = data.query('days_before_ipo >= 8 and days_before_ipo < 31')[cols_].groupby('ticker').agg(agg_dict).reset_index()
all_mean_data = data.query('days_before_ipo >= 31')[cols_].groupby('ticker').agg(agg_dict).reset_index()

one_day_count_data = data.query('days_before_ipo < 2').groupby('ticker')['views'].count().reset_index().rename({'views' : 'cnt'}, axis = 1)
one_week_count_data = data.query('days_before_ipo >= 2 and days_before_ipo < 8').groupby('ticker')['views'].count().reset_index().rename({'views' : 'cnt'}, axis = 1)
one_month_count_data = data.query('days_before_ipo >= 8 and days_before_ipo < 31').groupby('ticker')['views'].count().reset_index().rename({'views' : 'cnt'}, axis = 1)
all_count_data = data.query('days_before_ipo >= 31').groupby('ticker')['views'].count().reset_index().rename({'views' : 'cnt'}, axis = 1)

In [26]:
ipo_cols = ['ticker', '1d_price', '1m_price', '3m_price', '6m_price']

agg_data = pd.merge(ipo_data[ipo_cols], one_day_mean_data, on = 'ticker', how = 'left')
agg_data = pd.merge(agg_data, one_week_mean_data, on = 'ticker', how = 'left',  suffixes=('_one_day', '_one_week'))
agg_data = pd.merge(agg_data, one_month_mean_data, on = 'ticker', how = 'left')
agg_data = pd.merge(agg_data, all_mean_data, on = 'ticker', how = 'left',  suffixes=('_one_month', '_all'))


agg_data = pd.merge(agg_data, one_day_count_data, on = 'ticker', how = 'left')
agg_data = pd.merge(agg_data, one_week_count_data, on = 'ticker', how = 'left',  suffixes=('_one_day', '_one_week'))
agg_data = pd.merge(agg_data, one_month_count_data, on = 'ticker', how = 'left')
agg_data = pd.merge(agg_data, all_count_data, on = 'ticker', how = 'left',  suffixes=('_one_month', '_all'))

agg_data

,ticker,1d_price,1m_price,3m_price,6m_price,views_one_day,forwards_one_day,days_before_ipo_one_day,text_len_one_day,price_one_day,...,views_div_by_days_all,forwards_div_by_days_all,text_len_div_by_days_all,positive_score_div_by_days_all,views_x_positive_div_by_days_all,forwards_x_positive_div_by_days_all,cnt_one_day,cnt_one_week,cnt_one_month,cnt_all
0,KLVZ,-0.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN
1,DIAS,0.400,NaN,NaN,NaN,41663.000000,40.823529,1.0,1065.176471,19.45,...,756.524609,1.033877,41.739141,-1.708445e-02,756.524609,1.033877,17.0,8.0,5.0,10.0
2,DELI,0.068,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MGKL,-0.052,-0.01,NaN,NaN,21171.000000,13.000000,1.0,1069.000000,28.35,...,273.298385,0.249648,21.037274,-6.059701e-03,273.298385,0.249648,1.0,1.0,1.0,16.0
4,SVCB,0.087,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,560.280862,0.682263,20.390731,-3.252296e-03,560.280862,0.682263,NaN,21.0,9.0,96.0
5,UGLD,0.038,0.15,0.46,NaN,19357.000000,9.000000,1.0,966.000000,22.98,...,363.669549,0.927240,13.616916,1.429148e-04,363.669549,0.927240,1.0,1.0,3.0,15.0
6,EUTR,-0.108,-0.06,0.07,NaN,23250.500000,28.500000,1.0,761.000000,22.48,...,264.859704,0.724690,19.572883,1.039474e-03,264.859704,0.724690,2.0,5.0,14.0,22.0
7,HNFG,-0.007,-0.27,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,...,70.040850,0.123999,5.649689,-8.217717e-04,70.040850,0.123999,NaN,NaN,NaN,102.0
8,ASTR,0.400,0.51,0.57,NaN,NaN,NaN,NaN,NaN,NaN,...,346.223481,0.862561,21.784741,-7.976167e-04,346.223481,0.862561,NaN,48.0,100.0,666.0
9,CARM,0.050,-0.07,-0.16,-0.27,NaN,NaN,NaN,NaN,NaN,...,95.472794,0.103369,7.590531,-1.173661e-03,95.472794,0.103369,NaN,NaN,NaN,85.0


In [27]:
path = '../data/final/agg_data.csv'
agg_data.to_csv(path)